## 1. Imports

In [2]:
#importing libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from econml.dr import DRLearner
import warnings
from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix,roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import shap

## 2. Import Data




In [6]:
# Importing Data
X_train=pd.read_csv('/content/CausalAIdata/X_train.csv')
X_test=pd.read_csv('/content/CausalAIdata/X_test.csv')
y_train=pd.read_csv('/content/CausalAIdata/y_train.csv')
y_test=pd.read_csv('/content/CausalAIdata/y_test.csv')

In [7]:
# Check for missing values
if X_train.isnull().sum().any():
    print("There are missing values in the dataset.")
else:
    print("No missing values found in the dataset.")

No missing values found in the dataset.


In [8]:
X_train.columns

Index(['AnyHealthcare_1', 'HvyAlcoholConsump_1', 'Fruits_1', 'GenHlth_2',
       'GenHlth_3', 'GenHlth_4', 'GenHlth_5', 'DiffWalk_1',
       'HeartDiseaseorAttack_1', 'Education_2', 'Education_3', 'Education_4',
       'Education_5', 'Education_6', 'Sex_1', 'Stroke_1', 'Veggies_1',
       'CholCheck_1', 'Age_2', 'Age_3', 'Age_4', 'Age_5', 'Age_6', 'Age_7',
       'Age_8', 'Age_9', 'Age_10', 'Age_11', 'Age_12', 'Age_13',
       'PhysActivity_1', 'NoDocbcCost_1', 'HighChol_1', 'HighBP_1',
       'MentHlth_1', 'Income_2', 'Income_3', 'Income_4', 'Income_5',
       'Income_6', 'Income_7', 'Income_8', 'PhysHlth_1', 'Smoker_1', 'BMI'],
      dtype='object')

In [9]:
X_test.shape
# X train: 195052 , x test: 34422
# y train: 195052 , y test: 34422

(34422, 45)

## 3. Metrics functions

In [12]:
# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Function to evaluate metrics
def evaluate_metrics(y_true, y_pred, y_scores):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = specificity_score(y_true, y_pred)
    sensitivity = recall  # Sensitivity is the same as recall
    auc = roc_auc_score(y_true, y_scores)
    return precision, recall, specificity, sensitivity, auc

# 4. Cross-Validation code

In [13]:
# 5-fold Cross-Validation
def cross_validate_model(model, train_X, train_y, metrics_fn, cv=5):
    skf = StratifiedKFold(n_splits=cv)
    metrics_list = []

    for train_index, val_index in skf.split(train_X, train_y):
        #X_train, X_val = train_X.iloc[train_index], train_X.iloc[val_index]
        X_train_fold, X_val_fold = train_X.iloc[train_index], train_X.iloc[val_index]
        #y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]
        y_train_fold, y_val_fold = train_y.iloc[train_index], train_y.iloc[val_index]

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        y_scores = model.predict_proba(X_val_fold)[:, 1]

        metrics_list.append(metrics_fn(y_val_fold, y_pred, y_scores))

    metrics_df = pd.DataFrame(metrics_list, columns=["Precision", "Recall", "Specificity", "Sensitivity", "AUC"])
    return metrics_df.mean(axis=0)

# 5. Train LR Model

In [14]:
# Build LR model
LR_model = LogisticRegression(max_iter=1000)
LR_metrics = cross_validate_model(LR_model, X_train, y_train, evaluate_metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [15]:
LR_metrics

,0
Precision,0.560556
Recall,0.147861
Specificity,0.979028
Sensitivity,0.147861
AUC,0.808759


In [17]:
# save model
import joblib
import os
model_dir = "../model"
joblib.dump(LR_model,  "LR_model.joblib")

['LR_model.joblib']

# 6. Train LighGBM model

In [21]:
  # build the lightgbm model
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(
    objective='binary',         # Objective for binary classification
    boosting_type='gbdt',       # Gradient Boosting Decision Trees
    num_leaves=31,              # Maximum number of leaves in one tree
    learning_rate=0.05,         # Learning rate (step size shrinkage)
    n_estimators=100,           # Number of boosting iterations (trees)
    max_depth=10,               # Maximum tree depth
    min_child_samples=10,       # Minimum number of samples per leaf
    min_child_weight=5.0,       # Minimum sum of instance weights in a child node
    random_state=42             # Ensures reproducibility
)

In [22]:
lgb_metrics = cross_validate_model(lgb_model, X_train, y_train, evaluate_metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 23865, number of negative: 132176
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029914 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 163
[LightGBM] [Info] Number of data points in the train set: 156041, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152941 -> initscore=-1.711721
[LightGBM] [Info] Start training from score -1.711721


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 23865, number of negative: 132176
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029452 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 163
[LightGBM] [Info] Number of data points in the train set: 156041, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152941 -> initscore=-1.711721
[LightGBM] [Info] Start training from score -1.711721


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 23866, number of negative: 132176
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030516 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 164
[LightGBM] [Info] Number of data points in the train set: 156042, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152946 -> initscore=-1.711680
[LightGBM] [Info] Start training from score -1.711680


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 23866, number of negative: 132176
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039095 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 164
[LightGBM] [Info] Number of data points in the train set: 156042, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152946 -> initscore=-1.711680
[LightGBM] [Info] Start training from score -1.711680


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 23866, number of negative: 132176
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029509 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 164
[LightGBM] [Info] Number of data points in the train set: 156042, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152946 -> initscore=-1.711680
[LightGBM] [Info] Start training from score -1.711680


In [23]:
lgb_metrics

,0
Precision,0.586338
Recall,0.142096
Specificity,0.981867
Sensitivity,0.142096
AUC,0.813029


In [25]:
# save model
import joblib
import os
model_dir = "../model"
joblib.dump(lgb_model,  "LighGBM_model.pkl")

['LighGBM_model.pkl']

***________________________________________________######################___________________________________________________________________***

7. **Train SVC Model **

In [26]:

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Function to evaluate metrics
def evaluate_metrics(y_true, y_pred, y_scores):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = specificity_score(y_true, y_pred)
    sensitivity = recall  # Sensitivity is the same as recall
    auc = roc_auc_score(y_true, y_scores)
    return {"Precision": precision, "Recall": recall, "Specificity": specificity,
            "Sensitivity": sensitivity, "AUC": auc}

# Modified cross-validation function
def cross_validate_model2(model, train_X, train_y, cv=5):
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    all_y_true = []
    all_y_pred = []
    all_y_scores = []

    for train_index, val_index in skf.split(train_X, train_y):
        X_train_fold, X_val_fold = train_X.iloc[train_index], train_X.iloc[val_index]
        y_train_fold, y_val_fold = train_y.iloc[train_index], train_y.iloc[val_index]

        # Train model
        model.fit(X_train_fold, y_train_fold)

        # Predict and store results
        y_pred = model.predict(X_val_fold)
        y_scores = model.predict_proba(X_val_fold)[:, 1]

        all_y_true.extend(y_val_fold)
        all_y_pred.extend(y_pred)
        all_y_scores.extend(y_scores)

    # Calculate metrics outside the loop
    metrics = evaluate_metrics(all_y_true, all_y_pred, all_y_scores)
    return metrics


In [27]:
# Build SVC Model
SVM_model= SVC(gamma='auto',probability=True)


In [ ]:
# Perform cross-validation
SVM_metrics = cross_validate_model2(SVM_model, X_train, y_train)

# Print metrics
print("Cross-Validated Metrics:")
for metric, value in SVM_metrics.items():
    print(f"{metric}: {value:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# SVC model fitting
#SVM_metrics = cross_validate_model(SVM_model, X_train, y_train, evaluate_metrics)

In [ ]:
#SVM_metrics

In [ ]:
import joblib
import os
model_dir = "../model"
joblib.dump(SVM_model,"SVC_model.joblib")

In [ ]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8492533844634246
